# Random Coefficients Logit Tutorial with the Automobile Data

In [1]:
import pyblp
import numpy as np
import pandas as pd 

pyblp.options.digits = 2
pyblp.options.verbose = False
pyblp.__version__

'0.8.1'

In this tutorial, we'll use data from :ref:`references:Berry, Levinsohn, and Pakes (1995)` to solve the paper's automobile problem.


## Application of Random Coefficients Logit with the Automobile Data

This tutorial is similar to the [fake cereal tutorial](nevo.ipynb), but exhibits some other features of pyblp:

- Incorporating a supply side into demand estimation.
- Allowing for simple price-income demographic effects.
- Calculating clustered standard errors.


### Loading the Data

We'll use [pandas](https://pandas.pydata.org/) to load two sets of data:

1. `product_data`, which contains prices, shares, and other product characteristics.
2. `agent_data`, which contains draws from the distribution of heterogeneity.

In [2]:
product_data = pd.read_csv(pyblp.data.BLP_PRODUCTS_LOCATION)
product_data.head()

,market_ids,clustering_ids,car_ids,firm_ids,region,shares,prices,hpwt,air,mpd,...,supply_instruments11,supply_instruments12,supply_instruments13,supply_instruments14,supply_instruments15,supply_instruments16,supply_instruments17,supply_instruments18,supply_instruments19,supply_instruments20
0,1971,AMGREM71,129,15,US,0.001051,4.935802,0.528997,0,1.888146,...,2.024221,-0.251338,0.741272,0.820281,3.691881,0.566070,-2.328128,0.385197,0.648526,1.052606
1,1971,AMHORN71,130,15,US,0.000670,5.516049,0.494324,0,1.935989,...,2.026706,-0.205305,0.675468,1.004707,3.628727,0.605683,-2.268975,0.363954,0.643205,0.906429
2,1971,AMJAVL71,132,15,US,0.000341,7.108642,0.467613,0,1.716799,...,1.882594,-0.417634,0.946640,0.429942,4.001801,0.253461,-2.581003,-0.113763,0.770387,1.511333
3,1971,AMMATA71,134,15,US,0.000522,6.839506,0.426540,0,1.687871,...,1.846079,-0.445630,0.982922,0.392283,4.061161,0.183331,-2.626074,-0.253065,0.802037,1.583208
4,1971,AMAMBS71,136,15,US,0.000442,8.928395,0.452489,0,1.504286,...,1.760927,-0.610203,1.201745,-0.085373,4.340853,-0.043107,-2.785686,-0.556135,0.931611,2.177230


The `product_data` contains market IDs, product IDs, firm IDs, shares, prices, a number of product characteristics, and some pre-computed excluded instruments constructed with principal component analysis. The product IDs are called `clustering_ids` because they will be used to compute clustered standard errors. For more information about the instruments and the example data as a whole, refer to the :mod:`data` module.

The `agent_data` contains market IDs, integration weights $w_{it}$, integration nodes $\nu_{it}$, and demographics $d_{it}$. Here we use $I_t = 200$ equally weighted Monte Carlo draws per market.

In non-example problems, it is usually a better idea to use many more draws, or a more sophisticated :class:`Integration` configuration such as sparse grid quadrature.

In [3]:
agent_data = pd.read_csv(pyblp.data.BLP_AGENTS_LOCATION)
agent_data.head()

,market_ids,weights,nodes0,nodes1,nodes2,nodes3,nodes4,income
0,1971,0.005,1.764052,0.997845,0.945508,-1.962653,0.330046,4.172843
1,1971,0.005,0.400157,0.260081,0.422924,-0.834032,-0.000480,112.503416
2,1971,0.005,0.978738,0.925066,-1.175680,1.993837,0.818116,2.221875
3,1971,0.005,2.240893,1.476076,-0.204851,-0.666660,0.428214,0.814499
4,1971,0.005,1.867558,-1.879252,0.956495,0.976290,-2.503947,13.038725


### Setting up the Problem

Unlike the fake cereal problem, we won't absorb any fixed effects in the automobile problem, so the linear part of demand $X_1$ has more components. We also need to specify a formula for the random coefficients $X_2$, including a random coefficient on the constant, which captures correlation among all inside goods.

The primary new addition to the model relative to the fake cereal problem is that we add a supply side formula for product characteristics that contribute to marginal costs, $X_3$. The [patsy](https://patsy.readthedocs.io/en/stable/)-style formulas support functions of regressors such as the `log` function used below.

We stack the three product formulations in order: $X_1$, $X_2$, and $X_3$.

In [4]:
product_formulations = (
   pyblp.Formulation('1 + hpwt + air + mpd + space'),
   pyblp.Formulation('1 + prices + hpwt + air + mpd + space'),
   pyblp.Formulation('1 + log(hpwt) + air + log(mpg) + log(space) + trend')
)
product_formulations

(1 + hpwt + air + mpd + space,
 1 + prices + hpwt + air + mpd + space,
 1 + log(hpwt) + air + log(mpg) + log(space) + trend)

The original specification for the automobile problem includes the term $\log(y_i - p_j)$, in which $y$ is income and $p$ are prices. Instead of including this term, which gives rise to a host of numerical problems, we'll follow :ref:`references:Berry, Levinsohn, and Pakes (1999)` and use its first-order linear approximation, $p_j / y_i$. 

The agent formulation for demographics, $d$, includes a column of $1 / y_i$ values, which we'll interact with $p_j$. To do this, we will treat draws of $y_i$ as demographic variables.

In [5]:
agent_formulation = pyblp.Formulation('0 + I(1 / income)')
agent_formulation

I(1 / income)

As in the cereal example, the :class:`Problem` can be constructed by combining the `product_formulations`, `product_data`, `agent_formulation`, and `agent_data`. We'll also choose the functional form of marginal costs $c_{jt}$. A linear marginal cost specification is the default setting, so we'll need to use the `costs_type` argument of :meth:`Problem` to employ the log-linear specification used by :ref:`references:Berry, Levinsohn, and Pakes (1995)`.

In [6]:
problem = pyblp.Problem(product_formulations, product_data, agent_formulation, agent_data, costs_type='log')
problem

Dimensions:
 T    N     F    I     K1    K2    K3    D    MD    MS 
---  ----  ---  ----  ----  ----  ----  ---  ----  ----
20   2217  26   4000   5     6     6     1    18    27 

Formulations:
       Column Indices:            0          1       2       3          4         5  
-----------------------------  --------  ---------  ----  --------  ----------  -----
 X1: Linear Characteristics       1        hpwt     air     mpd       space          
X2: Nonlinear Characteristics     1       prices    hpwt    air        mpd      space
X3: Log Cost Characteristics      1      log(hpwt)  air   log(mpg)  log(space)  trend
       d: Demographics         1/income                                              

The problem outputs a table of dimensions:

- $T$ denotes the number of markets.
- $N$ is the length of the dataset (the number of products across all markets).
- $F$ denotes the number of firms.
- $I = \sum_t I_t$ is the total number of agents across all markets (200 draws per market times 20 markets).
- $K_1$ is the number of linear demand characteristics.
- $K_2$ is the number of nonlinear demand characteristics.
- $K_3$ is the number of linear supply characteristics.
- $D$ is the number of demographic variables.
- $M_D$ is the number of demand instruments, including exogenous regressors.
- $M_S$ is the number of supply instruments, including exogenous regressors.

The formulations table describes all four formulas for linear characteristics, nonlinear characteristics, cost characteristics, and demographics.

### Solving the Problem

The only remaining decisions are:

- Choosing $\Sigma$ and $\Pi$ starting values, $\Sigma_0$ and $\Pi_0$.
- Potentially choosing bounds for $\Sigma$ and $\Pi$.

The decisions we will use are:

- Use published estimates as our starting values in $\Sigma_0$.
- Interact the inverse of income, $1 / y_i$, only with prices, and use the published estimate on $\log(y_i - p_j)$ as our starting value for $\alpha$ in $\Pi_0$.
- Bound $\Sigma_0$ to be positive since it is a diagonal matrix where the diagonal consists of standard deviations.
- Constrain the $p_j / y_i$ coefficient to be negative. Specifically, we'll use a bound that's slightly smaller than zero because when the parameter is exactly zero, there are matrix inversion problems with estimating marginal costs.

When using a routine that supports bounds, :class:`Problem` chooses some default bounds. These bounds are often very wide, so it's usually a good idea to set your own more restrictive bounds.

In [7]:
initial_sigma = np.diag([3.612, 0, 4.628, 1.818, 1.050, 2.056])
initial_pi = np.c_[[0, -43.501, 0, 0, 0, 0]]
sigma_bounds = (
   np.zeros_like(initial_sigma),
   np.diag([100, 0, 100, 100, 100, 100])
)
pi_bounds = (
   np.c_[[0, -100, 0, 0, 0, 0]],
   np.c_[[0, -0.1, 0, 0, 0, 0]]
)

Note that there are only 5 nonzeros on the diagonal of $\Sigma$, which means that we only need 5 columns of integration nodes to integrate over these 5 dimensions of unobserved heterogeneity. Indeed, `agent_data` contains exactly 5 columns of nodes. If we were to ignore the $\log(y_i - p_j)$ term (by not configuring $\Pi$) and include a term on prices in $\Sigma$ instead, we would have needed 6 columns of integration nodes in our `agent_data`.

A downside of the log-linear marginal costs specification is that nonpositive estimated marginal costs can create problems for the optimization routine when computing $\log c(\hat{\theta})$. We'll use the `costs_bounds` argument to bound marginal costs from below by a small number. 

Finally, as in the original paper, we'll use `W_type` and `se_type` to cluster by product IDs, which were specified as `clustering_ids` in `product_data`.

In [8]:
results = problem.solve(
    initial_sigma,
    initial_pi,
    sigma_bounds=sigma_bounds,
    pi_bounds=pi_bounds,
    costs_bounds=(0.001, None),
    W_type='clustered',
    se_type='clustered'
)
results

Problem Results Summary:
GMM   Objective    Projected    Reduced Hessian  Reduced Hessian  Clipped  Weighting Matrix  Covariance Matrix
Step    Value    Gradient Norm  Min Eigenvalue   Max Eigenvalue    Costs   Condition Number  Condition Number 
----  ---------  -------------  ---------------  ---------------  -------  ----------------  -----------------
 2    +1.5E-01     +3.2E-09        +8.5E-05         +7.8E-02         2         +9.7E+05          +9.1E+07     

Cumulative Statistics:
Computation  Optimization   Objective   Fixed Point  Contraction
   Time       Iterations   Evaluations  Iterations   Evaluations
-----------  ------------  -----------  -----------  -----------
 00:08:55         60           71          20751        63650   

Nonlinear Coefficient Estimates (Robust SEs Adjusted for 999 Clusters in Parentheses):
Sigma:      1        prices      hpwt        air         mpd        space     |   Pi:     1/income 
------  ----------  --------  ----------  ----------  -----

There are some discrepancies between our results and the original paper. Our instruments are different, we use different agent data, and the first-order linear approximation to the $\ln(y_i - p_j)$ term is different as well.